In [2]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
result = model('sample.png')
result


image 1/1 c:\githome\wn7_dl_rep\sample.png: 640x608 3 persons, 43.3ms
Speed: 9.2ms preprocess, 43.3ms inference, 60.3ms postprocess per image at shape (1, 3, 640, 608)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted p

In [3]:
# 바운딩박스, 라벨 출력
result = model('red_panda.jpg')
result[0].boxes[0].conf[0]


image 1/1 c:\githome\wn7_dl_rep\red_panda.jpg: 352x640 3 bears, 33.6ms
Speed: 1.5ms preprocess, 33.6ms inference, 2.6ms postprocess per image at shape (1, 3, 352, 640)


tensor(0.9443, device='cuda:0')

In [4]:
import cv2 
img = cv2.imread('red_panda.jpg')
img

array([[[ 65, 154,  45],
        [ 72, 155,  52],
        [ 76, 149,  57],
        ...,
        [ 12,  16,  21],
        [  6,  10,  15],
        [ 28,  32,  37]],

       [[ 82, 171,  62],
        [ 74, 157,  54],
        [ 70, 143,  51],
        ...,
        [  6,  10,  15],
        [  0,   4,   9],
        [ 23,  27,  32]],

       [[ 76, 164,  58],
        [ 68, 151,  49],
        [ 67, 141,  47],
        ...,
        [  6,   9,  14],
        [  0,   3,   8],
        [ 22,  25,  30]],

       ...,

       [[112, 136,  94],
        [143, 171, 128],
        [157, 191, 150],
        ...,
        [200, 198, 198],
        [202, 200, 200],
        [204, 202, 202]],

       [[111, 132,  99],
        [124, 149, 115],
        [165, 198, 161],
        ...,
        [188, 186, 186],
        [183, 181, 181],
        [196, 194, 194]],

       [[135, 155, 126],
        [121, 145, 113],
        [165, 196, 163],
        ...,
        [178, 176, 176],
        [171, 169, 169],
        [183, 181, 181]]

In [6]:
for box in result[0].boxes:
    class_id = box.cls[0]
    class_name = model.names[class_id.item()]
    
    # 신뢰도 점수
    conf = float(box.conf[0])
    
    # 바운딩 박스
    coords = box.xyxy[0].tolist()
    x1, y1, x2, y2 = map(int, coords)
    
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(img, f'{class_name} {conf:.4f}',
                (x1,y1-10), cv2.FONT_HERSHEY_PLAIN,
                1, (0,255,0),1)
    
# end-for
cv2.imshow('yolov8 object detection', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('detect_result.jpg', img)

True

# Segmentation

In [19]:
from ultralytics import YOLO

model = YOLO("yolov8n-seg.pt")
result = model('street.jpg')
result[0].show()


image 1/1 c:\githome\wn7_dl_rep\street.jpg: 512x640 17 persons, 3 buss, 2 backpacks, 1 handbag, 30.7ms
Speed: 3.2ms preprocess, 30.7ms inference, 32.9ms postprocess per image at shape (1, 3, 512, 640)


In [ ]:
import numpy as np

img_new = cv2.imread('street.jpg')
overlay = img_new.copy() # 마스크용

for box, mask in zip(result[0].boxes, result[0].masks):
    
    polygon = mask.xy[0].astype(np.int32)
    cv2.fillPoly(overlay, [polygon], (0, 255, 0))
    
    class_id = box.cls[0]
    class_name = model.names[class_id.item()]
    
    # 신뢰도 점수
    conf = float(box.conf[0])
    
    # 바운딩 박스
    coords = box.xyxy[0].tolist()
    x1, y1, x2, y2 = map(int, coords)
    
    cv2.rectangle(img_new, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(img_new, f'{class_name} {conf:.4f}',
                (x1,y1-10), cv2.FONT_HERSHEY_PLAIN,
                1, (0,255,0),1)

# img + overlay combine and print
alpha = 0.5
final_img = cv2.addWeighted(overlay, alpha, img_new, 1-alpha, 0)

# 최종출력
cv2.imshow('yolov8 object segmentation', final_img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('detect_result.jpg', final_img)

True

# 분류 (생략)
# 자세

In [25]:
pose_model = YOLO("yolov8n-pose.pt")
pose_result = pose_model('V.jpg')
img = cv2.imread('V.jpg')

kp = pose_result[0].keypoints

for man_kp in kp:
    for x, y in man_kp.xy[0]:
        if x>0 and y>0:
            cv2.circle(img, (int(x), int(y)), 3, (255, 0, 0), -1)
            
cv2.imshow('yolov8 object pose estimation', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('pose_result.jpg', img)


image 1/1 c:\githome\wn7_dl_rep\V.jpg: 640x640 1 person, 28.5ms
Speed: 5.1ms preprocess, 28.5ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)


True

In [26]:
pose_model = YOLO("yolov8n-pose.pt")
pose_result = pose_model('RV.jpg')
img = cv2.imread('RV.jpg')

kp = pose_result[0].keypoints

for man_kp in kp:
    for x, y in man_kp.xy[0]:
        if x>0 and y>0:
            cv2.circle(img, (int(x), int(y)), 3, (255, 0, 0), -1)
            
cv2.imshow('yolov8 object pose estimation', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('pose_result.jpg', img)


image 1/1 c:\githome\wn7_dl_rep\RV.jpg: 448x640 3 persons, 33.9ms
Speed: 2.7ms preprocess, 33.9ms inference, 5.9ms postprocess per image at shape (1, 3, 448, 640)


True

# 모델 구조 + 가중치

In [29]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
data_path = 'datasets/Rock Paper Scissors/data.yaml'
model.train(data=data_path, epochs=5, imgsz=640)

Ultralytics 8.3.191  Python-3.10.18 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=datasets/Rock Paper Scissors/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0,

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001E8109F8490>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          

In [35]:
result = model.predict('sample.png')
result[0].show()


image 1/1 c:\githome\wn7_dl_rep\sample.png: 640x608 1 Paper, 1 Rock, 2 Scissorss, 22.4ms
Speed: 3.9ms preprocess, 22.4ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 608)


In [34]:
model = YOLO("runs/detect/train/weights/best.pt")
data_path = 'datasets/Rock Paper Scissors/data.yaml'
metrics = model.val(data=data_path)
metrics

Ultralytics 8.3.191  Python-3.10.18 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 615.5171.1 MB/s, size: 29.1 KB)
val: Scanning C:\githome\wn7_dl_rep\datasets\Rock Paper Scissors\valid\labels.cache... 576 images, 238 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 576/576 575904.4it/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 36/36 10.5it/s 3.4s0.1s
                   all        576        400      0.756      0.706      0.773      0.546
                 Paper        132        139      0.731       0.54      0.633      0.425
                  Rock        121        141      0.909      0.778      0.887      0.637
              Scissors        116        120      0.627        0.8      0.798      0.577
Speed: 0.5ms preprocess, 2.9ms inference, 0.0ms loss, 0.7ms postprocess pe

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001E8CEBCBBB0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          

In [36]:
# yolov8n.pt, best.pt의 성능 비교, sample.png
data_path = 'datasets/Rock Paper Scissors/data.yaml'

metrics = model.val(data=data_path)

Ultralytics 8.3.191  Python-3.10.18 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
val: Fast image access  (ping: 0.00.0 ms, read: 420.5230.7 MB/s, size: 27.3 KB)
val: Scanning C:\githome\wn7_dl_rep\datasets\Rock Paper Scissors\valid\labels.cache... 576 images, 238 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 576/576 575630.0it/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 36/36 10.8it/s 3.3s0.2s
                   all        576        400      0.756      0.706      0.773      0.546
                 Paper        132        139      0.731       0.54      0.633      0.425
                  Rock        121        141      0.909      0.778      0.887      0.637
              Scissors        116        120      0.627        0.8      0.798      0.577
Speed: 0.4ms preprocess, 2.6ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect\val2


In [37]:
metrics

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001E8C070D900>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          

In [1]:
# best_parameters.yaml
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

# 하이퍼파라미터 탐색 수행
model.tune(
    data='datasets/Rock Paper Scissors/data.yaml',
    epochs=5,
    imgsz=640,
    iterations=5,
    val=True
)

Tuner: Initialized Tuner instance with 'tune_dir=runs\detect\tune3'
Tuner:  Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/5 with hyperparameters: {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'bgr': 0.0, 'mosaic': 1.0, 'mixup': 0.0, 'cutmix': 0.0, 'copy_paste': 0.0}
Saved runs\detect\tune3\tune_scatter_plots.png
Saved runs\detect\tune3\tune_fitness.png

Tuner: 1/5 iterations complete  (271.08s)
Tuner: Results saved to runs\detect\tune3
Tuner: Best fitness=0.71376 observed at iteration 1
Tuner: Best fitness metrics are {'metrics/precision(B)': 0.92147, 'metrics/recall(B)': 0.85509, 'metrics/mAP50(B)': 0.91551, 'metrics/mAP50-95(B)': 0.69134, 'val/box_loss': 0.92063, 'v